<center><img src="https://i.imgur.com/zRrFdsf.png" width="700"></center>

# Data Formatting (categorical)

<a target="_blank" href="https://colab.research.google.com/github/CienciaDeDatosEspacial/code_and_data/blob/main/Formatting_II_Categorical.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

In this formatting tutorial we will see the categorical case.


Let's get [some data](https://en.wikipedia.org/wiki/List_of_freedom_indices):

In [2]:
%reset
import pandas as pd

link='https://en.wikipedia.org/wiki/List_of_freedom_indices'
freeDFs=pd.read_html(link,flavor='bs4',match='w',attrs={'class':"wikitable"})

# how many tables?
len(freeDFs)

Once deleted, variables cannot be recovered. Proceed (y/[n])? y


URLError: <urlopen error [SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: certificate has expired (_ssl.c:1002)>

In [ ]:
#is this one?
freeDFs[0]

Then, you want the second table:

In [ ]:
allFree=freeDFs[1]
allFree.head()

Cleaning column names:

In [ ]:
allFree.columns

Is this a good alternative?

In [ ]:
allFree.columns.str.replace(r"\W|\d","",regex=True)

You might prefer this:

In [ ]:
NewNames=['Country', 'Freedom', 'FreedomScore', 'EconomicFreedom',
       'EconomicFreedomScore', 'PressFreedom', 'PressFreedomScore', 'Democracy', 'DemocracyScore']
allFree.columns=NewNames

Let's check data types:

In [ ]:
allFree.info()

Let's clean all the leading/trailing space in every cell:

In [ ]:
# this code breaks if applied to numeric columns
allFree=allFree.apply(lambda x: x.str.strip())

Do we have unique country names?

In [ ]:
len(allFree.Country)==len(pd.unique(allFree.Country))

In [ ]:
allFree.head()

You have categorical and numerical columns. Would you prefer this look:

In [ ]:
#non scores

allFree.columns[~allFree.columns.str.contains("score",case=False)]

In [ ]:
# non scores as index
allFree.set_index(allFree.columns[~allFree.columns.str.contains("score",case=False)].to_list())

In [ ]:
# reset index
allFree.set_index(allFree.columns[~allFree.columns.str.contains("score",case=False)].to_list()).reset_index(drop=False)

In [ ]:
#Then
allFree=allFree.set_index(allFree.columns[~allFree.columns.str.contains("score",case=False)].to_list()).reset_index(drop=False)

In [ ]:
allFree.head()

Let's pay attention to the categorical columns:

In [ ]:
[list(allFree[c].sort_values().unique()) for c in allFree.columns[1:5]]

You wanted to check for mistakes like ['free', 'not free', 'partly free', 'Free']. As we can see, the cells are clean.

Now, let's turn the values into **ordinal** categories. Remember that the worst, best and middle values should be comparable:

In [ ]:
mapper1 = {'not free': 1,'partly free': 3, 'free':5 }
allFree.Freedom.replace(mapper1,inplace=True)

mapper2 = {'repressed':1, 'mostly unfree':2,'moderately free':3, 'mostly free':4, 'free':5}
allFree.EconomicFreedom.replace(mapper2,inplace=True)


mapper3 = {'very serious':1,'difficult':2, 'problematic':3,
           'satisfactory':4,'good':5}
allFree.PressFreedom.replace(mapper3,inplace=True)

mapper4 = {'authoritarian regime':1, 'hybrid regime':2,'flawed democracy':4,'full democracy':5}
allFree.Democracy.replace(mapper4,inplace=True)

In [ ]:
# what we have so far
allFree

Let's see the data types:

In [ ]:
#check types:
allFree.info()

We could turn those floats into integers:

In [ ]:
# what about
allFree[allFree.columns[1:5]]=allFree.iloc[:,1:5].apply(lambda x: x.astype('Int64'))

In [ ]:
#then
allFree.info()

However, these are NOT yet **ordinal**. Let's see how the process is:

In [ ]:
from pandas.api.types import CategoricalDtype

order = CategoricalDtype(categories=[1,2,3,4,5], ordered=True)
allFree.iloc[:,1:5].apply(lambda x:x.astype(order)).info()

That is what you will get. Let me put those values in new columns:

1. Create new column names

In [ ]:
# create some new names:
newNames=allFree.columns[1:5]+'_ord'
#see
newNames

2. Create new data columns (using the new names):

In [ ]:
allFree[newNames]=allFree.iloc[:,1:5].apply(lambda x:x.astype(order))

You have this:

In [ ]:
allFree.info()

Notice the ordinal levels:

In [ ]:
allFree.EconomicFreedom_ord

You may want to rename them:

In [ ]:
# This is how it looks:


ordCats={1:'veryLow',2:'low',3:'medium',4:'good',5:'veryGood'}

turnToOrdinal= lambda x:x.cat.rename_categories(ordCats)

allFree.iloc[:,9:].apply(turnToOrdinal)

In [ ]:
# making the changes:

allFree[newNames]=allFree.iloc[:,9:].apply(turnToOrdinal)

# see
allFree.head(10)

Let's keep this last result, but this time let me show you the use of **pickle** format:

In [ ]:
#saving in "data" folder

import os 

allFree.to_csv(os.path.join("data","allFree.csv"),index=False ) # as usual
allFree.to_pickle(os.path.join("data","allFree.pkl") )

In [ ]:
#reading

dfPickle=pd.read_pickle(os.path.join("data","allFree.pkl") )  
dfCSV=pd.read_csv(os.path.join("data","allFree.csv") )  

Now, notice the difference when you have categorical data:

In [ ]:
dfPickle.info()

In [ ]:
dfCSV.info()

In [ ]:
# the file kept the data type
dfPickle.Democracy_ord

In [ ]:
# the file did not keep the data type
dfCSV.Democracy_ord